<a href="https://colab.research.google.com/github/Junjaee/Study/blob/main/Python_Data_Analytics_Library/08_%EB%B6%84%EC%84%9D%EC%98%88%EC%A0%9C4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib as mpl
import matplotlib.font_manager as fm
fm.FontProperties(fname = "맑은 고딕")
mpl.rcParams["axes.unicode_minus"] = False # 음수표시

import seaborn as sns

plt.style.use("fivethirtyeight")

import warnings
warnings.filterwarnings("ignore")


import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import requests
# from selenium import webdriver
import urllib.parse

from bs4 import BeautifulSoup

# 1. 시카고 샌드위치 맛집 분석

- https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/


#### 1) 데이터 준비

In [20]:
url = "https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/"

# 로봇이 아님을 증명
headers = {"User-Agent" : "Mozilla/5.0"}
html = requests.get(url, headers = headers)

soup = BeautifulSoup(html.text, "html.parser")
soup

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<title>The 50 Best Sandwiches in Chicago – Chicago Magazine</title>
<style type="text/css">			.heateorSssInstagramBackground{background:radial-gradient(circle at 30% 107%,#fdf497 0,#fdf497 5%,#fd5949 45%,#d6249f 60%,#285aeb 90%)}
						div.heateor_sss_horizontal_sharing i.heateorSssInstagramBackground{background:#000!important;}div.heateor_sss_standard_follow_icons_container i.heateorSssInstagramBackground{background:#000;}
										.heateor_sss_horizontal_sharing .heateorSssSharing,.heateor_sss_standard_follow_icons_container .heateorSssSharing{
							background-color: #000;
							color: #fff;
						border-width: 0px;
			border-style: solid;
			border-color: transparent;
		}
				.heateor_sss_horizontal_sharing .heateorSssTCBackground{
			color:#666;
		}
				.heateor_sss_horizontal_sharing .heateorSssSharing:ho

In [21]:
print(soup.find_all("div", "sammy"))

[<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">2</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/"><b>Fried Bologna</b><br/>
Au Cheval<br/>
<em>Read more</em> </a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">3</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/"><b>Woodland Mushroom</b><br/>
Xoco<br/>
<em>Read more</em> </a></div>
</div>, <div class="sammy" style="position: relative;">
<div class="sammyRank">4</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-R

In [22]:
len(soup.find_all("div", "sammy"))

50

In [23]:
print(soup.find_all("div", "sammy")[0])

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>


#### 2) parsing

In [24]:
##### 50개 데이터 중 첫번째 데이터 가져오기
one = soup.find_all("div", "sammy")[0]
print(type(one))
print(one)

<class 'bs4.element.Tag'>
<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>


In [25]:
##### 첫번째 데이터 중에서 랭킹 가져오기

one.find(class_ = "sammyRank").get_text()

'1'

In [44]:
##### 메뉴 이름과 가게 이름 가져오기

one_list = one.find(class_ = "sammyListing").get_text()


# 정규표현식으로 split
import re

print(re.split(('\n|\r\n'), one_list)[0])
print(re.split(('\n|\r\n'), one_list)[1])

BLT
Old Oak Tap


In [45]:
##### a 태그에서 href 값 가져오기

one.find("a")["href"]

# 링크 경로의 형식이 동일하지 않기 때문에 똑같이 맞춰준다

base = "https://www.chicagomag.com"

print(base + url)

urllib.parse.urljoin(base, url)


https://www.chicagomag.comhttps://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/


'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'

#### 3) 각 데이터 수집

- rank
- main_menu
- cafe_name
- ignore


In [47]:
rank = []
main_menu = []
cafe_name = []
readmore = []
url_base = "https://www.chicagomag.com"

cafes = soup.find_all('div','sammy')

for item in cafes:
    rank.append(item.find(class_='sammyRank').get_text())
    
    tmp = item.find(class_='sammyListing').get_text()
    main_menu.append(re.split(('\n|\r\n'), tmp)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp)[1])
                         
    readmore.append(urllib.parse.urljoin(url_base, item.find('a')['href']))

In [48]:
len(rank), len(main_menu), len(cafe_name), len(readmore)

(50, 50, 50, 50)

In [49]:
rank[:5]
main_menu[:5]

['BLT', 'Fried Bologna', 'Woodland Mushroom', 'Roast Beef', 'PB&L']

In [50]:
##### DataFrame에 저장

data = {"Rank" : rank, "Menu" : main_menu, "Cafe" : cafe_name, "ReadMore" : readmore}
df = pd.DataFrame(data)
df.head()

,Rank,Menu,Cafe,ReadMore
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...


#### 4) Checkpoint

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


',Rank,Menu,Cafe,ReadMore\n0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/\n1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/\n2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/\n3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Als-Deli-Roast-Beef/\n4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Publican-Quality-Meats-PB-L/\n5,6,Belgian Chicken Curry Salad,Hendrickx Belgian Bread Crafter,https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Hendrickx-Belgian-Bread-Crafter-Belgian-Chicken-Curry-Salad/\n6,7,Lobster Roll,Acadia,https://www.chicagomag.com/Chicago-Magazine/November-2012/Be

In [53]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/Study/Python Data Analytics Library/data/sandwiches.csv", encoding = "utf-8")

In [54]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Study/Python Data Analytics Library/data/sandwiches.csv", index_col = 0)
df.head()

,Rank,Menu,Cafe,ReadMore
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...


#### 5) 상세 페이지 데이터 가져오기

In [70]:
url = df["ReadMore"][0]
html = requests.get(url, headers = {"User-Agent" : "Mozilla/5.0"})

soup = BeautifulSoup(html.text, "html.parser")
soup

detail = soup.find("p", "addy").get_text()
print(detail)
detail_split = detail.split()
print(detail_split)

# 가격 가져오기
print(detail_split[0])
print(detail_split[0][:-1])


$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com
['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']
$10.
$10


In [66]:
' '.join(detail_split[1:-2])

'2109 W. Chicago Ave.,'

In [ ]:
##### 각 데이터 수집 : 가격, 주소
##### hupyter notebook에서 상태 진행바를 보여주기
  # conda install -c conda-forge tqdm

In [73]:
from tqdm import tqdm_notebook

price = []
address = []

for n in tqdm_notebook(df.index):
    html = requests.get(df["ReadMore"][n], headers = {"User-Agent" : "Mozilla/5.0"})
    soup = BeautifulSoup(html.text, 'html.parser')
    
    detail = soup.find('p', 'addy').get_text()
    
    price.append(detail.split()[0][:-1])
    address.append(' '.join(detail.split()[1:-2]))

In [72]:
len(rank), len(main_menu), len(cafe_name), len(readmore)

(50, 50, 50, 50)

In [75]:
rank[:5]
main_menu[:5]
address[:5]

['2109 W. Chicago Ave.,',
 '800 W. Randolph St.,',
 '445 N. Clark St.,',
 '914 Noyes St., Evanston,',
 '825 W. Fulton Mkt.,']

In [76]:
#### 두 개의 정보를 DataFrame에 추가

df["Price"] = price
df["Address"] = address
df.head()

,Rank,Menu,Cafe,ReadMore,Price,Address
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


#### 6) Checkpoint

In [77]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/Study/Python Data Analytics Library/data/sandwiches_final.csv", encoding = "utf-8")

In [78]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Study/Python Data Analytics Library/data/sandwiches_final.csv", index_col = 0)
df.head()

,Rank,Menu,Cafe,ReadMore,Price,Address
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"2109 W. Chicago Ave.,"
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...,$9,"800 W. Randolph St.,"
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...,$9.50,"445 N. Clark St.,"
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...,$9.40,"914 Noyes St., Evanston,"
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...,$10,"825 W. Fulton Mkt.,"


#### 7) 맛집의 위치를 지도에 표시

In [ ]:
import folium
import googlemaps